네이버 영화평 감상 분석(TfidfVectorizer)

In [11]:
import numpy as np 
import pandas as pd
from konlpy.tag import Okt

In [12]:
train_df = pd.read_csv('../00.data/naverMovie/train.tsv', sep='\t')
test_df = pd.read_csv('../00.data/naverMovie/test.tsv', sep='\t')

In [13]:
train_df['document'].head(5)

0                                    아 더빙 진짜 짜증나네요 목소리
1                           흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 솔직히 재미는 없다평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
Name: document, dtype: object

Tokenizer 함수 정의

In [14]:
okt = Okt()
def tw_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [16]:
tvecter = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)


In [17]:
%time tvecter.fit(train_df.document)

Wall time: 9min 40s


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x0000027871F41B80>)

In [18]:
%time X_train_tvect = tvecter.transform(train_df['document'])

Wall time: 10min 5s


In [19]:
%time X_test_tvect = tvecter.transform(test_df['document'])

Wall time: 3min 17s


In [20]:
y_train = train_df.label.values
y_test = test_df.label.values

LogisticRegression 학습/예측/평가

In [22]:
lr_clf = LogisticRegression(C=3.5)
lr_clf.fit(X_train_tvect, y_train)
pred = lr_clf.predict(X_test_tvect)
accuracy_score(y_test, pred)

0.8590876620063271

실제테스트

In [26]:
review1 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ '
review2 ='원더우먼은 재밌다. 인정합니다. 맨오브스틸3 언제 나올려나...'

In [27]:
import re
review= re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review1)
review_tvect = tvecter.transform([review]) 
#tvecter 위에서 이미 토큰화가 되어있다. tvecter = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
pred = lr_clf.predict(review_tvect)
pred[0]

0

In [29]:
review =re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review2)
review_tvect = tvecter.transform([review])
pred = lr_clf.predict(review_tvect)
pred[0]

1

In [30]:
reviews = ['진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ',
           '이런 사랑영화가 다시 나올 수 있을까?']

In [31]:
reviews = list(map(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", x), reviews))
print(reviews)

['진짜 개노잼이다 편이랑 같은 감독맞나러닝타임도 길어서 개지루함 ㄹㅇ', '이런 사랑영화가 다시 나올 수 있을까']


In [32]:
review_tvect = tvecter.transform(reviews)
pred = lr_clf.predict(review_tvect)
pred[0], pred[1]

(0, 1)